In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum().any()

In [ ]:
test.isnull().any().sum()

In [ ]:
train['time'] = pd.to_datetime(train['time'],format='%Y-%m-%d %H:%M:%S')
test['time'] = pd.to_datetime(test['time'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
#train['Date'] = train['time'].dt.date
train['weekday'] = train['time'].dt.weekday
#train['year'] = train['time'].dt.year
train['Day'] = train['time'].dt.day
train['month'] = train['time'].dt.month
train['day of year'] = train['time'].dt.dayofyear
train['hour'] = train['time'].dt.hour
#train['second'] = train['time'].dt.second
train['qtr']=train['time'].dt.quarter


In [ ]:
train.head()

In [ ]:
#test['Date'] = test['time'].dt.date
test['weekday'] = test['time'].dt.weekday
#test['year'] = test['time'].dt.year
test['Day'] = test['time'].dt.day
test['month'] = test['time'].dt.month
test['day of year'] = test['time'].dt.dayofyear
test['hour'] = test['time'].dt.hour
test['qtr']=test['time'].dt.quarter


In [ ]:
train['y'].unique()

In [ ]:
train.congestion.unique()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
plt.figure(figsize=(18, 6))
sns.countplot(x = "congestion", data = train)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
plt.figure(figsize=(13, 5))
sns.violinplot(x = 'direction', y = 'congestion', data = train)

In [ ]:
df = train.drop(['row_id'], axis = 1)
from sklearn.preprocessing import LabelEncoder, StandardScaler
sc = StandardScaler()
le = LabelEncoder()
df['direction']=le.fit_transform(df['direction'])
plt.figure(figsize=(10,7))
corr = df.corr()
sns.heatmap(corr, cmap = 'coolwarm',  annot = True)

In [ ]:
train = train.drop(['row_id', 'time'], axis= 1)
test = test.drop(['row_id', 'time'], axis= 1)
#train['direction']=le.fit_transform(train['direction'])
#test['direction']=le.fit_transform(test['direction'])


In [ ]:
#Q1 = train.quantile(0.2)
#Q3 = train.quantile(0.8)
#IQR = Q3 - Q1

In [ ]:
#train1 = train[~((train < (Q1 - 1.5 * IQR)) |(train > (Q3 + 1.5 * IQR))).any(axis=1)]
#train1.shape

In [ ]:
ohe_train = pd.get_dummies(train, columns = ['direction', 'x', 'y'])
ohe_test = pd.get_dummies(test, columns = ['direction', 'x', 'y'])

In [ ]:
ohe_train.head()

In [ ]:
X = ohe_train.drop(['congestion'], axis = 1)
y = ohe_train['congestion']
X = sc.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 44)

In [ ]:
import xgboost as xg
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
xgb = xg.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1,
             max_delta_step=0, max_depth=6, min_child_weight=1, 
             monotone_constraints='()', n_estimators=2000, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=2364, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='auto',
             validate_parameters=1, verbosity=0)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
r2_score = metrics.r2_score(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
print('RMSE:', RMSE, '\n', "R2Score:", '\n', r2_score)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(early_stopping=True)
br = BaggingRegressor()
import catboost as cb
import shap
cat = cb.CatBoostRegressor(loss_function='RMSE', random_state = 44,  iterations=2000,learning_rate=0.1,depth=6,l2_leaf_reg=7)
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingRegressor
hgb = HistGradientBoostingRegressor(learning_rate=0.1, max_iter=1000, random_state=50)
dt = DecisionTreeRegressor(max_depth=None, min_samples_split=2,random_state=43)
rf = RandomForestRegressor(n_estimators=700, criterion='mse', max_depth=7, random_state = 41)
gbr = GradientBoostingRegressor()

In [ ]:
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
r2_score = metrics.r2_score(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
print('RMSE:', RMSE, '\n', "R2Score:", '\n', r2_score)

In [ ]:
test1 = sc.transform(ohe_test)
y_pred = xgb.predict(test1).round(3)
print(y_pred)

In [ ]:
test_prediction_ohe = pd.DataFrame(y_pred, columns=['congestion'])
test_prediction_ohe.head()

In [ ]:
ID = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
test_id = ID['row_id']
ID = pd.DataFrame(test_id, columns=['row_id'])

In [ ]:
result = pd.concat([ID,test_prediction_ohe], axis=1)
result

In [ ]:
result.to_csv('submission_xgb1_ohe.csv',index =False)